In [1]:
import cv2
from ultralytics import YOLO
import time
import mss
import numpy as np
import pyautogui
import keyboard
from pynput.mouse import Button, Controller

# Загрузка модели
# model = YOLO("yolov8n.pt")  # Загрузка официальной модели
model = YOLO(r"runs\detect\train2\weights\best.pt")  # Загрузка пользовательской модели

# Задание области захвата экрана
monitor = {"top": 362, "left": 551, "width": 801, "height": 481}

mouse = Controller()

prev_frame_time = 0
new_frame_time = 0

# Флаг для завершения цикла
exit_flag = False

def on_press_q(e):
    global exit_flag
    if e.name == 'q':
        exit_flag = True

# Установка обработчика нажатия клавиши 'q'
keyboard.on_press(on_press_q)
time.sleep(1)
with mss.mss() as sct:
    while not exit_flag:
        # Захват области экрана
        sct_img = sct.grab(monitor)
        
        # Преобразование изображения в формат numpy
        frame = np.array(sct_img)
        
        # Изменение цветового пространства с BGRA на BGR
        frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

        # Запись времени начала обработки кадра
        start_time = time.time()

        # Детектирование объектов на кадре
        results = model(frame)

        # Отображение результатов на кадре
        annotated_frame = results[0].plot()
        
        # Вывод координат xyxy для каждого обнаруженного объекта
        for r in results:
            for box in r.boxes:
                xyxy = box.xyxy  # Предполагается, что xyxy - атрибут объекта box
                if xyxy.shape[1] == 4:  # Проверка, что xyxy содержит 4 элемента
                    x_min, y_min, x_max, y_max = xyxy[0, 0], xyxy[0, 1], xyxy[0, 2], xyxy[0, 3]
                    print(f"x_min: {x_min:.2f}, y_min: {y_min:.2f}, x_max: {x_max:.2f}, y_max: {y_max:.2f}")
                else:
                    print("Invalid box coordinates format")
                
                abs_x_min = monitor["left"] + x_min
                abs_y_min = monitor["top"] + y_min
                abs_x_max = monitor["left"] + x_max
                abs_y_max = monitor["top"] + y_max
                
                x_center = (abs_x_min + abs_x_max) / 2
                y_center = (abs_y_min + abs_y_max) / 2
                mouse.position = (x_center, y_center)
                mouse.click(Button.left)

        # Расчет и отображение FPS
        new_frame_time = time.time()
        fps = 1 / (new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        cv2.putText(annotated_frame, f'{int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Вывод кадра в окно
        cv2.imshow('YOLO Object Detection', annotated_frame)

        # Печать времени обработки кадра (опционально, для анализа производительности)
        print(f'Time per frame: {time.time() - start_time} seconds')

        # Проверка нажатия клавиши 'q' для выхода
        if exit_flag:
            break

# Освобождение ресурсов
cv2.destroyAllWindows()



0: 384x640 30 <aim>s, 188.5ms
Speed: 7.0ms preprocess, 188.5ms inference, 1044.4ms postprocess per image at shape (1, 3, 384, 640)
x_min: 474.09, y_min: 416.53, x_max: 532.27, y_max: 476.50
x_min: 596.16, y_min: 390.41, x_max: 653.84, y_max: 448.16
x_min: 109.76, y_min: 268.65, x_max: 161.15, y_max: 344.72
x_min: 292.77, y_min: 371.48, x_max: 346.14, y_max: 426.12
x_min: 13.66, y_min: 402.84, x_max: 61.46, y_max: 453.03
x_min: 545.37, y_min: 216.92, x_max: 581.40, y_max: 253.02
x_min: 650.09, y_min: 235.79, x_max: 701.71, y_max: 286.48
x_min: 318.55, y_min: 36.65, x_max: 367.25, y_max: 86.08
x_min: 144.29, y_min: 350.32, x_max: 181.80, y_max: 387.80
x_min: 709.63, y_min: 298.82, x_max: 747.93, y_max: 337.62
x_min: 170.31, y_min: 202.55, x_max: 204.08, y_max: 235.81
x_min: 128.72, y_min: 145.99, x_max: 159.73, y_max: 178.10
x_min: 429.94, y_min: 91.76, x_max: 497.74, y_max: 146.71
x_min: 543.74, y_min: 269.44, x_max: 572.68, y_max: 300.50
x_min: 764.22, y_min: 277.69, x_max: 778.44, y_